In [ ]:
from datacube.utils.dask import start_local_dask
from dep_tools.searchers import PystacSearcher
from dep_tools.loaders import OdcLoader
from dep_tools.namers import LocalPath
from dep_tools.writers import LocalDsCogWriter

from dep_tools.grids import PACIFIC_GRID_10, PACIFIC_EPSG

from src.utils import GeoMADAWSSentinel2Processor, get_grid

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# Optionally set up a local dask cluster
client = start_local_dask(threads_per_worker=64, n_workers=1, mem_safety_margin="2GB")
client.dashboard_link

In [ ]:
# Study site configuration
item_id = "63,20"  # West viti levu
# item_id = "65,22"  # Near the anti-meridian
# item_id = "66,22"  # Right of the anti-meridian

datetime = "2024"

# And get the study site
tile_index = tuple(int(i) for i in item_id.split(","))
geobox = PACIFIC_GRID_10.tile_geobox(tile_index).zoom_out(10)
geobox.explore()

In [ ]:
# Search for data
searcher = PystacSearcher(
    catalog="https://earth-search.aws.element84.com/v1",
    collections=["sentinel-2-c1-l2a"],
    datetime=datetime,   
)

items = searcher.search(area=geobox)
print(f"Found {len(items)} items")

In [ ]:
# Set up a data loader
loader = OdcLoader(
    # bands=["SCL", "B02", "B03", "B04", "B05", "B06", "B07", "B08", "B8A", "B11", "B12"],
    bands=["cloud", "red", "blue", "green"],
    chunks=dict(time=1, x=4096, y=4096),
)

# Run the load process, which is lazy-loaded
input_data = loader.load(items, areas=geobox)
input_data

In [ ]:
# Set up a data processor
processor = GeoMADAWSSentinel2Processor(
    geomad_options=dict(
        work_chunks=(2000, 2000),
        num_threads=10,
        maxiters=100,
    ),
    filters=[("closing", 5), ("opening", 5)],
    mask_cloud_percentage=20,
    min_timesteps=5,
    drop_vars = ["cloud"]
)

# Plan the processing. Still lazy-loaded
output_data = processor.process(input_data)
output_data

In [ ]:
output_data.odc.explore(vmin=0, vmax=2000)

In [ ]:
# This is the target path
dep_path = LocalPath(
    local_folder="data",
    sensor="s2",
    dataset_id="geomad",
    version="0.1.0",
    time=datetime,
)

# Set up a writer and write out the files
writer = LocalDsCogWriter(itempath=dep_path)

# THIS DOESN'T WORK!
# out_files = writer.write(output_data, item_id)

In [ ]:
# Make sure written files are readable
stac_path = writer.itempath.path(item_id, ext=".stac-item.json")

item = Item.from_file(stac_path)
item.validate()